In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import csv
from IPython.display import display, clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
lol_watcher = LolWatcher('RGAPI-c3c3d5b7-e533-47d6-ab6b-675b962dfe14')

In [19]:
def crawler_details():
    my_region = 'euw1'
    crawed = pd.read_csv('test.csv')

    test = []
    waiting = games['gameId']
    count = len(waiting)
    for i in waiting:
        try:
            if str(i) not in crawed['gameId'].to_list():
                test.append(lol_watcher.match.by_id(my_region, i))
            clear_output(wait=True)
            print("Waiting to complete: ", count)
            count -= 1
        except KeyboardInterrupt:
            break
        except Exception as e:
            with open('log.txt', 'a+', newline='') as file:
                file.write(str(e))
            break

    with open('test.csv', 'a+', newline='') as output_file:
        print("Writen")
        dict_writer = csv.DictWriter(output_file, test[0].keys())
    #     dict_writer.writeheader()
        dict_writer.writerows(test)
        with open('log.txt', 'a+', newline='') as file:
                file.write("\nWritten, lacks ")
                file.write(str(count))
                file.write("\n")
    return count

In [21]:
count = crawler_details()
while (count > 0):
    count = crawler_details()

Waiting to complete:  1
Writen


In [14]:
crawed = pd.read_csv('test.csv')
clean = crawed.drop_duplicates(subset=['gameId'], keep='first', inplace=False)
clean.to_csv('game_details.csv', encoding='utf8', index=False)

In [26]:
def crawler_timelines():
    my_region = 'euw1'
    flag = False
    try:
        crawed = pd.read_csv('test.csv')
        exists = crawed['gameId'].to_list()
    except FileNotFoundError:
        exists = []
        flag = True

    test = []
    waiting = games['gameId']
    count = len(waiting)
    for i in waiting:
        try:
            if i not in exists:
                print("Request")
                timeline = lol_watcher.match.timeline_by_match(my_region, i)
                timeline['gameId'] = i
                test.append(timeline)
            clear_output(wait=True)
            print("Waiting to complete: ", count)
            count -= 1
        except KeyboardInterrupt:
            break
        except Exception as e:
            with open('log.txt', 'a+', newline='') as file:
                file.write(str(e))
            break

    with open('test.csv', 'a+', newline='') as output_file:
        print("Writen")
        dict_writer = csv.DictWriter(output_file, test[0].keys())
        if flag:
            dict_writer.writeheader()
        dict_writer.writerows(test)
        with open('log.txt', 'a+', newline='') as file:
                file.write("\nWritten, lacks ")
                file.write(str(count))
                file.write("\n")
    return count

In [ ]:
count = crawler_timelines()
while (count > 0):
    count = crawler_timelines()

In [10]:
result = pd.read_csv('game_details.csv')

team = list(eval(result['teams'][0])[0].keys())
bluekeys = ["Blue " + s for s in team]
redkeys = ["Red " + s for s in team]
keys = ["gameId"] + bluekeys + redkeys
count = len(result['teams'])

test = pd.DataFrame(columns = keys)

for i, j in zip(result['teams'], result['gameId']):
    clear_output(wait=True)
    print("Waiting to complete: ", count)
    teams = eval(i)
    values = [str(j)] + list((teams)[0].values()) + list((teams)[1].values())
    res = {keys[i]: values[i] for i in range(len(keys))}
    test = test.append(res, ignore_index=True)
    count -= 1

test.to_csv('teams_details.csv', encoding='utf8', index=False)
print("Written")

Waiting to complete:  1
Written


In [63]:
count = {15:0, 20:0, 25:0, 30:0, 35:0, 40:0, 45:0}
for k, j in enumerate(timeline['frames']):
    frame = eval(j)
    for i in count.keys():
        if i < frame[-1]['timestamp'] / 60000:
            count[i] += 1
            clear_output(wait=True)
            print("Update Game", k, "ended at after", i, "mins:", count[i])

Update Game 9878 ended at after 20 mins: 8630


In [64]:
for i in count:
    print("Game ended at after", i, "mins:", count[i])

Game ended at after 15 mins: 9819
Game ended at after 20 mins: 8630
Game ended at after 25 mins: 6213
Game ended at after 30 mins: 3260
Game ended at after 35 mins: 1129
Game ended at after 40 mins: 249
Game ended at after 45 mins: 53


# Reform Data

In [ ]:
main = pd.read_csv('main.csv')
timeline = pd.read_csv('test.csv')
details = pd.read_csv('teams_details.csv')

In [ ]:
cols = list(main.keys())

In [ ]:
# calculer les données selon la structure de notre data set, 
# un paramètre entré signifie les x premiers mins du
def getCsv(mins):
    dictionnaire = []
    for numb, times in enumerate(timeline['frames']):
        clear_output(wait = True)
        print("Processing: Game", numb)
        
        times = eval(times)
        dic = {}
        for feature in cols:
            dic[feature] = 0
            
        dic['gameId'] = str(timeline['gameId'][numb])
        result = list(details.loc[details['gameId'] == dic['gameId']]['Blue win'])[0]
        print("result: ", result[0])
        dic['blueWins'] = 1 if result[0] == 'W' else 0
        b_level = 0
        r_level = 0
        
        try:
            for k,v in times[mins]['participantFrames'].items():
                #average level
                if(v['participantId'] <= 5 and  v['participantId'] >= 1):
                    b_level += v['level']
                    dic['blueTotalGold'] += v['totalGold']
                    dic['blueTotalExperience'] += v['xp']
                    dic['blueTotalMinionsKilled'] += v['minionsKilled']
                    dic['blueTotalJungleMinionsKilled'] += v['jungleMinionsKilled']
                else:
                    r_level += v['level']
                    dic['redTotalGold'] += v['totalGold']
                    dic['redTotalExperience'] += v['xp']
                    dic['redTotalMinionsKilled'] += v['minionsKilled']
                    dic['redTotalJungleMinionsKilled'] += v['jungleMinionsKilled']
                    
            dic['blueAvgLevel'] = b_level / 5
            dic['blueGoldDiff'] = dic['blueTotalGold'] - dic['redTotalGold']
            dic['blueExperienceDiff'] = dic['blueTotalExperience'] - dic['redTotalExperience']
            dic['blueCSPerMin'] = dic['blueTotalMinionsKilled']/mins
            dic['blueGoldPerMin'] = dic['blueTotalGold']/mins

            dic['redAvgLevel'] = r_level / 5
            dic['redGoldDiff'] = -dic['blueGoldDiff']
            dic['redExperienceDiff'] = -dic['blueExperienceDiff']
            dic['redCSPerMin'] = dic['redTotalMinionsKilled'] / mins
            dic['redGoldPerMin'] = dic['redTotalGold'] / mins
            first = False
            for j in range(mins+1):
                for i in times[j]['events']: 
                    if(i.get('type') == "WARD_PLACED" and i.get('creatorId') != 0):
                        if(i.get('creatorId') <= 5):
                            dic['blueWardsPlaced'] += 1
                        else :
                            dic['redWardsPlaced'] += 1
                    elif(i.get('type') == "WARD_KILL" and i.get('killerId') != 0):
                        if(i.get('killerId') <= 5):
                            dic['blueWardsDestroyed'] += 1
                        else :
                            dic['redWardsDestroyed'] += 1
                    elif(i.get('type') == "ELITE_MONSTER_KILL" and i.get('killerId') != 0):
                        if(i.get('killerId') <= 5):
                            dic['blueEliteMonsters'] += 1
                            if(i.get('monsterType') == "DRAGON"):
                                dic['blueDragons'] += 1
                            elif(i.get('monsterType') == "RIFTHERALD"):
                                dic['blueHeralds'] += 1
                        else :
                            dic['redEliteMonsters'] += 1
                            if(i.get('monsterType') == "DRAGON"):
                                dic['redDragons'] += 1
                            elif(i.get('monsterType') == "RIFTHERALD"):
                                dic['redHeralds'] += 1
                    elif(i.get('type') == "CHAMPION_KILL" and i.get('killerId') != 0):
                        if(i.get('killerId') <= 5):
                            if(not first):
                                dic['blueFirstBlood'] = 1
                            dic['blueKills'] += 1
                            dic['redDeaths'] += 1
                            dic['blueAssists'] += len(i.get('assistingParticipantIds'))
                        else :
                            if(not first):
                                dic['redFirstBlood'] = 1
                            dic['redKills'] += 1
                            dic['blueDeaths'] += 1
                            dic['redAssists'] += len(i.get('assistingParticipantIds'))
                        first=True
                    elif(i.get('type') == "BUILDING_KILL" and 
                         i.get('buildingType') == "TOWER_BUILDING" and 
                         i.get('killerId') != 0):
                        if(i.get('killerId') <= 5):
                            dic['blueTowersDestroyed'] += 1
                        else:
                            dic['redTowersDestroyed'] += 1
            dictionnaire.append(dic)
        except IndexError as e:
            continue
        except Exception as e:
            print(e)
            print(dic)
            return None
    return dictionnaire

In [ ]:
def new_csv(mins, test):
    with open('test_'+str(mins)+'mins.csv', 'a+', newline='') as output_file:
        writer = csv.DictWriter(output_file,test[0].keys())
        writer.writeheader()
        writer.writerows(test)
        with open('log.txt', 'a+', newline='') as file:
                file.write("\n")

In [ ]:
for i in [15, 20, 25, 30, 35]:
    test = getCsv(i)
    new_csv(i, test)

In [ ]:
def getdata(times, mins, cols):
    dic = {}
    for feature in cols:
        dic[feature] = 0
            
    b_level = 0
    r_level = 0

    for k,v in times[mins]['participantFrames'].items():
        #average level
        if(v['participantId'] <= 5 and  v['participantId'] >= 1):
            b_level += v['level']
            dic['blueTotalGold'] += v['totalGold']
            dic['blueTotalExperience'] += v['xp']
            dic['blueTotalMinionsKilled'] += v['minionsKilled']
            dic['blueTotalJungleMinionsKilled'] += v['jungleMinionsKilled']
        else:
            r_level += v['level']
            dic['redTotalGold'] += v['totalGold']
            dic['redTotalExperience'] += v['xp']
            dic['redTotalMinionsKilled'] += v['minionsKilled']
            dic['redTotalJungleMinionsKilled'] += v['jungleMinionsKilled']

    dic['blueAvgLevel'] = b_level / 5
    dic['blueGoldDiff'] = dic['blueTotalGold'] - dic['redTotalGold']
    dic['blueExperienceDiff'] = dic['blueTotalExperience'] - dic['redTotalExperience']
    dic['blueCSPerMin'] = dic['blueTotalMinionsKilled']/mins
    dic['blueGoldPerMin'] = dic['blueTotalGold']/mins

    dic['redAvgLevel'] = r_level / 5
    dic['redGoldDiff'] = -dic['blueGoldDiff']
    dic['redExperienceDiff'] = -dic['blueExperienceDiff']
    dic['redCSPerMin'] = dic['redTotalMinionsKilled'] / mins
    dic['redGoldPerMin'] = dic['redTotalGold'] / mins
    
    first = False
    for j in range(mins+1):
        for i in times[j]['events']: 
            if(i.get('type') == "WARD_PLACED" and i.get('creatorId') != 0):
                if(i.get('creatorId') <= 5):
                    dic['blueWardsPlaced'] += 1
                else :
                    dic['redWardsPlaced'] += 1
            elif(i.get('type') == "WARD_KILL" and i.get('killerId') != 0):
                if(i.get('killerId') <= 5):
                    dic['blueWardsDestroyed'] += 1
                else :
                    dic['redWardsDestroyed'] += 1
            elif(i.get('type') == "ELITE_MONSTER_KILL" and i.get('killerId') != 0):
                if(i.get('killerId') <= 5):
                    dic['blueEliteMonsters'] += 1
                    if(i.get('monsterType') == "DRAGON"):
                        dic['blueDragons'] += 1
                    elif(i.get('monsterType') == "RIFTHERALD"):
                        dic['blueHeralds'] += 1
                else :
                    dic['redEliteMonsters'] += 1
                    if(i.get('monsterType') == "DRAGON"):
                        dic['redDragons'] += 1
                    elif(i.get('monsterType') == "RIFTHERALD"):
                        dic['redHeralds'] += 1
            elif(i.get('type') == "CHAMPION_KILL" and i.get('killerId') != 0):
                if(i.get('killerId') <= 5):
                    if(not first):
                        dic['blueFirstBlood'] = 1
                    dic['blueKills'] += 1
                    dic['redDeaths'] += 1
                    dic['blueAssists'] += len(i.get('assistingParticipantIds'))
                else :
                    if(not first):
                        dic['redFirstBlood'] = 1
                    dic['redKills'] += 1
                    dic['blueDeaths'] += 1
                    dic['redAssists'] += len(i.get('assistingParticipantIds'))
                first=True
            elif(i.get('type') == "BUILDING_KILL" and 
                 i.get('buildingType') == "TOWER_BUILDING" and 
                 i.get('killerId') != 0):
                if(i.get('killerId') <= 5):
                    dic['blueTowersDestroyed'] += 1
                else:
                    dic['redTowersDestroyed'] += 1
    return dic

In [3]:
lol_watcher.match.timeline_by_match('euw1', 4519157822)

{'frames': [{'participantFrames': {'1': {'participantId': 5,
     'position': {'x': 560, 'y': 581},
     'currentGold': 500,
     'totalGold': 500,
     'level': 1,
     'xp': 0,
     'minionsKilled': 0,
     'jungleMinionsKilled': 0,
     'dominionScore': 0,
     'teamScore': 0},
    '2': {'participantId': 4,
     'position': {'x': 560, 'y': 361},
     'currentGold': 500,
     'totalGold': 500,
     'level': 1,
     'xp': 0,
     'minionsKilled': 0,
     'jungleMinionsKilled': 0,
     'dominionScore': 0,
     'teamScore': 0},
    '3': {'participantId': 2,
     'position': {'x': 351, 'y': 293},
     'currentGold': 500,
     'totalGold': 500,
     'level': 1,
     'xp': 0,
     'minionsKilled': 0,
     'jungleMinionsKilled': 0,
     'dominionScore': 0,
     'teamScore': 0},
    '4': {'participantId': 3,
     'position': {'x': 221, 'y': 471},
     'currentGold': 500,
     'totalGold': 500,
     'level': 1,
     'xp': 0,
     'minionsKilled': 0,
     'jungleMinionsKilled': 0,
     'domini